In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import pairwise_distances
import scipy.spatial

### Please provide the following information, so a song can be recommended:
(one of the given options or *None* should be chosen)

UserID = [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
       1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
       1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033,
       1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042]
DrivingStyle = ['relaxed driving', 'sport driving']
landscape = ['urban', 'mountains', 'country side', 'coast line']
mood = ['sad', 'lazy', 'active', 'happy']
naturalphenomena = ['night', 'morning', 'day time', 'afternoon']
RoadType = ['city', 'serpentine', 'highway']
sleepiness = ['sleepy', 'awake']
trafficConditions = ['traffic jam', 'lots of cars', 'free road']
weather = ['sunny', 'snowing', 'rainy', 'cloudy']

In [46]:
# ['UserID', 'DrivingStyle', 'landscape', 'mood', 'naturalphenomena',
#        'RoadType', 'sleepiness', 'trafficConditions', 'weather']
# input = [[1001], ['relaxed driving'], [None], ['lazy'], ['night'],
#         [None], [None], ['lots of cars'], ['rainy']]
input = {'DrivingStyle': ['relaxed driving'],
         'landscape': [None],
         'mood': ['lazy'],
         'naturalphenomena': ['night'],
         'RoadType': [None],
         'sleepiness': [None],
         'trafficConditions': ['lots of cars'],
         'weather': ['rainy'],
         'UserID': [1001]}

## 1. Read data

In [3]:
excel_path = "C:\\Users\\aneta\\Documents\\GitHub\\car-music-recommendations\\Music_InCarMusic\\Data_InCarMusic.xlsx"
df = pd.read_excel(excel_path, sheet_name="ContextualRating")
df.columns = df.columns.str.replace(' ', '')
df.drop_duplicates(inplace=True)
tracks_df = pd.read_excel(excel_path, sheet_name="Music Track")
tracks_df.columns = tracks_df.columns.str.replace(' ', '')

In [106]:
tracks_df

,id,album,artist,title,mp3url,description,imageurl,category_id
0,248,\N,B.B.King,The Thrill is Gone,Music/Blues/B.B.King - The Thrill is Gone.mp3,\N,img/music.jpg,1
1,250,\N,Mamie Smith,Crazy Blues,Music/Blues/Mamie Smith - Crazy Blues.mp3,\N,img/music.jpg,1
2,251,\N,Robert Johnson,Hellhound On My Trail,Music/Blues/Robert Johnson - Hellhound On My T...,\N,img/music.jpg,1
3,252,\N,T-Bone Walker,Stormy Monday,Music/Blues/T-Bone Walker - Stormy Monday.mp3,\N,img/music.jpg,1
4,253,\N,Antonio Vivaldi,Four Seasons,Music/Classical/Antonio Vivaldi - Four Seasons...,\N,img/music.jpg,2
...,...,...,...,...,...,...,...,...
134,759,\N,Fettes Brot,Jein (Radio Edit),Music/MusicLoad/Fettes_Brot--Jein__Radio_Edit_...,\N,http://a-static.musicload-shop.de/cov/m/75/88/...,8
135,761,\N,Lady GaGa,Paparazzi,Music/MusicLoad/Lady_GaGa--Paparazzi.mp3,\N,http://a-static.musicload-shop.de/cov/m/75/88/...,8
136,760,\N,Bon Jovi,Superman Tonight - Album Version,Music/MusicLoad/Bon_Jovi--Superman_Tonight_-_A...,\N,http://a-static.musicload-shop.de/cov/m/75/88/...,10
137,762,\N,DJ Gollum vs. Basslovers United,Narcotic,Music/MusicLoad/DJ_Gollum_vs__Basslovers_Unite...,\N,http://c-static.musicload-shop.de/cov/m/75/91/...,4


## 2. Create a similarity matrix

In [5]:
features_columns = df.columns.values
features_columns = np.delete(features_columns, 0)
features_columns = np.append(features_columns, 'UserID')
features_columns = features_columns[2:]
features_columns

array(['DrivingStyle', 'landscape', 'mood', 'naturalphenomena',
       'RoadType', 'sleepiness', 'trafficConditions', 'weather', 'UserID'],
      dtype=object)

In [6]:
dummies = pd.get_dummies(df, columns=features_columns)
dummies.insert(loc=0, column='id', value=np.arange(len(dummies)))
dummies

,id,ItemID,Rating,DrivingStyle_relaxed driving,DrivingStyle_sport driving,landscape_coast line,landscape_country side,landscape_mountains,landscape_urban,mood_active,...,UserID_1033,UserID_1034,UserID_1035,UserID_1036,UserID_1037,UserID_1038,UserID_1039,UserID_1040,UserID_1041,UserID_1042
0,0,715,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,267,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,294,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,259,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,674,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007,3933,716,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4008,3934,733,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4009,3935,682,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4010,3936,691,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
features = dummies.iloc[:, 3:]
features = features.set_index(dummies['id'])
features

,DrivingStyle_relaxed driving,DrivingStyle_sport driving,landscape_coast line,landscape_country side,landscape_mountains,landscape_urban,mood_active,mood_happy,mood_lazy,mood_sad,...,UserID_1033,UserID_1034,UserID_1035,UserID_1036,UserID_1037,UserID_1038,UserID_1039,UserID_1040,UserID_1041,UserID_1042
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3934,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3935,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
features_sparse = csr_matrix(features.fillna(0))

In [9]:
recommender = pairwise_distances(features_sparse, metric='cosine')
recommender_df = pd.DataFrame(recommender, index=features.index, columns=features.index)

## 3. Analyze user input, find most similar item that is top-rated

In [36]:
reference_df = dummies.drop(['id', 'ItemID', 'Rating'], axis=1)
reference_df

,DrivingStyle_relaxed driving,DrivingStyle_sport driving,landscape_coast line,landscape_country side,landscape_mountains,landscape_urban,mood_active,mood_happy,mood_lazy,mood_sad,...,UserID_1033,UserID_1034,UserID_1035,UserID_1036,UserID_1037,UserID_1038,UserID_1039,UserID_1040,UserID_1041,UserID_1042
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [54]:
input_df = pd.DataFrame(input)
input_dummies = pd.get_dummies(input_df)
userid_col_name = 'UserID_' + str(input_dummies.iloc[0]['UserID'])
input_dummies[userid_col_name] = 1
input_dummies = input_dummies.drop('UserID', axis=1)
input_dummies

,DrivingStyle_relaxed driving,mood_lazy,naturalphenomena_night,trafficConditions_lots of cars,weather_rainy,UserID_1001
0,1,1,1,1,1,1


In [55]:
missing_cols = np.setdiff1d(reference_df.columns.values, input_dummies.columns.values)
missing_cols

array(['DrivingStyle_sport driving', 'RoadType_city', 'RoadType_highway',
       'RoadType_serpentine', 'UserID_1002', 'UserID_1003', 'UserID_1004',
       'UserID_1005', 'UserID_1006', 'UserID_1007', 'UserID_1008',
       'UserID_1009', 'UserID_1010', 'UserID_1011', 'UserID_1012',
       'UserID_1013', 'UserID_1014', 'UserID_1015', 'UserID_1016',
       'UserID_1017', 'UserID_1018', 'UserID_1019', 'UserID_1020',
       'UserID_1021', 'UserID_1022', 'UserID_1023', 'UserID_1024',
       'UserID_1025', 'UserID_1026', 'UserID_1027', 'UserID_1028',
       'UserID_1029', 'UserID_1030', 'UserID_1031', 'UserID_1032',
       'UserID_1033', 'UserID_1034', 'UserID_1035', 'UserID_1036',
       'UserID_1037', 'UserID_1038', 'UserID_1039', 'UserID_1040',
       'UserID_1041', 'UserID_1042', 'landscape_coast line',
       'landscape_country side', 'landscape_mountains', 'landscape_urban',
       'mood_active', 'mood_happy', 'mood_sad',
       'naturalphenomena_afternoon', 'naturalphenomena_day time'

In [56]:
for col in missing_cols:
    input_dummies[col] = 0
input_dummies

,DrivingStyle_relaxed driving,mood_lazy,naturalphenomena_night,trafficConditions_lots of cars,weather_rainy,UserID_1001,DrivingStyle_sport driving,RoadType_city,RoadType_highway,RoadType_serpentine,...,naturalphenomena_afternoon,naturalphenomena_day time,naturalphenomena_morning,sleepiness_awake,sleepiness_sleepy,trafficConditions_free road,trafficConditions_traffic jam,weather_cloudy,weather_snowing,weather_sunny
0,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
ary = scipy.spatial.distance.cdist(reference_df, input_dummies, metric='euclidean')
best_fitted = reference_df[ary==ary.min()].iloc[:5][:].index.values
best_fitted

array([23, 24, 25, 26, 27], dtype=int64)

In [83]:
best_fitted_ratings = []
for item in best_fitted:
    best_fitted_ratings.append(dummies.iloc[item][2])
best_fitted_ratings

[2, 4, 3, 4, 4]

In [84]:
max_index = best_fitted_ratings.index(max(best_fitted_ratings))
max_index

1

In [85]:
best_choice = best_fitted[max_index]
best_choice

24

## 4. Find TOP10 items similar to the found item

In [104]:
top10_indexes = recommender_df[24].sort_values()[1:11].index.values
top10_indexes

array([25, 30, 32, 33, 34, 35, 36, 37, 38, 39], dtype=int64)

In [105]:
top10_itemid = []
for index in top10_indexes:
    top10_itemid.append(dummies.iloc[index][1])
top10_itemid

[674, 259, 267, 251, 715, 285, 674, 294, 279, 754]

In [108]:
# nie do wszystkich id przypisane są utwory - błąd
top10_songs = []
for itemid in top10_itemid:
    top10_songs.append(tracks_df['id' == itemid])
top10_songs

KeyError: False